In [6]:
# Analysis of min-accesses required capacity

"""
TileFlow and LoopTree can only have one trunk, and all intermediate tensors
must be backed by the split.

Thus, two choices:

Choice 1:  Untiled fusion, weights can be tiled

DRAM
GLB -- Intermediates()
  Intra-Einsum mapping

  
Choice 2: Tiled fusion, weights cannot be tiled

DRAM
GLB -- Weights
for p of last layer
GLB -- Intermediates()
  Intra-Einsum mapping
"""

from fastfusion.frontend import Workload
from fastfusion.frontend.workload.isl import get_tensor_size

workload = Workload.from_yaml('workloads/mobilenet_28.workload.yaml')
einsums = workload.einsums

# In choice 1, long pole Einsum for capacity is Dwise1
choice_1_looptree_long_pole = sum(
    get_tensor_size(workload, tensor)
    for tensor in einsums['Dwise1'].tensor_names & workload.intermediate_tensor_names
)
print('Choice 1 LoopTree long pole:', choice_1_looptree_long_pole)

choice_1_tileflow_long_pole = sum(
    get_tensor_size(workload, tensor)
    for tensor in einsums['Dwise1'].tensor_names  # TileFlow has to be even
)
print('Choice 1 TileFlow long pole:', choice_1_tileflow_long_pole)


# In choice 2, long pole is keeping all weights
choice_2_long_pole = sum(
    get_tensor_size(workload, tensor)
    for tensor in ['WA0', 'WAB0', 'WB0', 'WA1', 'WAB1', 'WB1']
)

choice_2_long_pole += sum(
    get_tensor_size(workload, tensor)
    for tensor in ['T0', 'TA0']
) * 4/28

choice_2_long_pole += sum(
    get_tensor_size(workload, tensor)
    for tensor in ['TB0', 'T1', 'TA1']
) * 2/28

choice_2_long_pole += sum(
    get_tensor_size(workload, tensor)
    for tensor in ['TB1', 'T2']
) * 1/28 
print('Choice 2 long pole:', choice_2_long_pole)

INFO        Loading yaml file workloads/mobilenet_28.workload.yaml
INFO        Found top key workload in workloads/mobilenet_28.workload.yaml
WARNING     Trying to parse a single element dictionary as a Workload. 


Choice 1 LoopTree long pole: 301056
Choice 1 TileFlow long pole: 301824
Choice 2 long pole: 74016.0


In [2]:
loopforest_long_pole = sum(
    get_tensor_size(workload, tensor)
    for tensor in ['WA0', 'WAB0', 'WB0']
)/192 + sum(
    get_tensor_size(workload, tensor)
    for tensor in ['T0', 'T1']
)

loopforest_long_pole

37689.0

In [8]:
from snowcat_experiments import get_sims_with_cache

# mappings_7_loopforest = get_sims_with_cache(arch_name='snowcat', workload_name='mobilenet_7')
# mappings_7_looptree = get_sims_with_cache(arch_name='snowcat', workload_name='mobilenet_7', tagger_name='one_split')
# mappings_7_tileflow = get_sims_with_cache(arch_name='snowcat_even', workload_name='mobilenet_7', tagger_name='one_split')

mappings_28_loopforest = get_sims_with_cache(arch_name='snowcat', workload_name='mobilenet_28')
# mappings_28_looptree = get_sims_with_cache(arch_name='snowcat', workload_name='mobilenet_28', tagger_name='one_split')
mappings_28_tileflow = get_sims_with_cache(arch_name='snowcat_even', workload_name='mobilenet_28', tagger_name='one_split')

# mappings_14_loopforest = get_sims_with_cache(arch_name='snowcat', workload_name='mobilenet_14')
# mappings_14_looptree = get_sims_with_cache(arch_name='snowcat', workload_name='mobilenet_14', tagger_name='one_split')
# mappings_14_tileflow = get_sims_with_cache(arch_name='snowcat_even', workload_name='mobilenet_14', tagger_name='one_split')

Loaded final results from cache results/data/mobilenet_28.snowcat.None.pkl
Loaded final results from cache results/data/mobilenet_28.snowcat_even.one_split.pkl


In [9]:
from fastfusion.frontend import Workload
from fastfusion.visualization.interactive import plotly_show

workload = Workload.from_yaml('workloads/mobilenet_28.workload.yaml')
plotly_show(mappings_28_loopforest.data, "RESOURCE_GlobalBuffer_LEVEL_0", "metric_Energy", logscales=True, einsum_names=workload.einsum_names)

INFO        Loading yaml file workloads/mobilenet_28.workload.yaml
INFO        Found top key workload in workloads/mobilenet_28.workload.yaml
WARNING     Trying to parse a single element dictionary as a Workload. 


    'data': [{'line': {'shape': 'hv'},
              'marker': {'symbol': 'circl…

In [ ]:
plotly_show(mappings_28_looptree.data, "RESOURCE_GlobalBuffer_LEVEL_0", "metric_Energy", logscales=True, einsum_names=workload.einsum_names)

In [5]:
plotly_show(mappings_28_tileflow.data, "RESOURCE_GlobalBuffer_LEVEL_0", "metric_Energy", logscales=True, einsum_names=workload.einsum_names)

    'data': [{'line': {'shape': 'hv'},
              'marker': {'symbol': 'circl…

In [22]:
from geomean import continuous_gm
import numpy as np

data = mappings_28_loopforest.data[["RESOURCE_GlobalBuffer_LEVEL_0", "metric_Energy"]].to_numpy().T
looptree_baseline = mappings_28_looptree.data[["RESOURCE_GlobalBuffer_LEVEL_0", "metric_Energy"]].to_numpy().T
looptree_baseline[0,-1] = 91488
print(continuous_gm(looptree_baseline, data))

NameError: name 'mappings_28_looptree' is not defined

In [26]:
data = mappings_28_loopforest.data[["RESOURCE_GlobalBuffer_LEVEL_0", "metric_Energy"]].to_numpy().T
sort_mask = np.argsort(data[0,:])
data = data[:,sort_mask]

tileflow_baseline = mappings_28_tileflow.data[["RESOURCE_GlobalBuffer_LEVEL_0", "metric_Energy"]].to_numpy().T
sort_mask = np.argsort(tileflow_baseline[0,:])
tileflow_baseline = tileflow_baseline[:,sort_mask]

# tileflow_baseline[0,-1] = 91488
print(continuous_gm(tileflow_baseline, data))

(np.float64(1.3453674382492102), np.float64(1.9780439121756488))
